In [ ]:
import json
import pandas as pd
import ast
from statistics import mean
from time import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=""
)

system_prompt = """
당신은 감사문서 기반 질문 문서 유사도 평가 전문가입니다.
아래 다섯 가지 기준을 사용해 질문(question)과 문서(document)가 유사한지 판단하십시오.
판정은 매우 엄격하게 수행하며 True는 기준을 명확히 충족할 때만 선택하십시오.

[평가 기준]

1. 주제 일치(Topic Match)
   질문과 문서가 다루는 감사 분야가 세부 분야 수준에서 완전히 동일할 때만 True로 판단합니다.
   출장비 계약 수의계약 등 동일한 분야여야 하며 큰 범주가 비슷하거나 내부통제 같은 일반 단어만 겹치는 경우는 False입니다.
   또한 질문에 특정 기간 또는 특정 연도 조건이 포함된 경우 문서의 날짜(date) 값이 그 기간과 정확히 일치해야 하며 날짜가 맞지 않으면 주제가 동일하더라도 False입니다.

2. 세부쟁점 일치(Sub Issue Match)
   질문이 요구하는 핵심 쟁점이 문서에서 다루는 구체적 문제와 직접적으로 일치할 때만 True입니다.
   예를 들어 질문이 출장비 증빙 누락 문제를 묻는 경우 문서에도 증빙 누락 문제나 관련 부적정 지급 쟁점이 포함되어 있어야 합니다.
   같은 분야라도 문제 포인트가 다르면 False입니다.

3. 사건 메커니즘 유사(Case Mechanism Match)
   문제 발생 과정과 사건 전개 방식이 질문과 문서에서 동일한 경우에만 True입니다.
   절차 미준수로 인한 부당 지급과 같은 단계적 구조가 일치해야 합니다.
   결과만 비슷하거나 원리 구조가 다르면 False입니다.

4. 위반행위 패턴 유사(Violation Pattern Match)
   부정행위의 유형이 질문과 문서 양쪽에서 동일할 때 True입니다.
   허위 청구 부당 지급 규정 미준수 등 위반 패턴이 완전히 일치해야 합니다.
   유형이 다르면 False입니다.

5. 원인 구조 유사(Cause Pattern Match)
   문제의 근본 원인이 질문과 문서에서 동일할 때만 True입니다.
   내부통제 미흡 관리 감독 소홀 규정 미비 등 원인 체계가 같아야 합니다.
   개인 일탈이나 고의적 비위처럼 다른 구조라면 False입니다.

[출력 형식]
아래 JSON 형식으로만 출력하십시오.

{{
  "topic_match": true/false,
  "subtopic_match": true/false,
  "case_structure_match": true/false,
  "violation_pattern_match": true/false,
  "cause_pattern_match": true/false
}}

추가 설명 이유 해석 문장은 절대 포함하지 마십시오.
"""

user_prompt = """
[Question]
{question}

[Document]
{document}
"""

evaluation_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", user_prompt)
])

def convert_matches_to_score(match_dict):
    count = sum(match_dict.values())

    if count >= 4:
        return 1.0
    elif count >= 2:
        return 0.6
    elif count == 1:
        return 0.2
    else:
        return 0.0

df = pd.read_csv("./data/retrieval_data.csv", encoding="utf-8")

df["documents"] = df["documents"].apply(ast.literal_eval)

total_rows = len(df)
print(f"총 {total_rows}개 row 평가 시작")

all_doc_sims = []
all_mean_sims = []

start_time = time()

for i, (_, row) in enumerate(df.iterrows()):
    question = row["question"]
    documents = row["documents"]

    doc_scores = []

    for doc in documents:
        doc_text = "\n".join([f"{k}: {v}" for k, v in doc.items()])

        prompt = evaluation_prompt.format(
            question=question,
            document=doc_text
        )
        response = llm.invoke(prompt)

        result = json.loads(response.content)

        score = convert_matches_to_score(result)
        doc_scores.append(score)

    all_doc_sims.append(doc_scores)
    all_mean_sims.append(mean(doc_scores))

    if (i + 1) % 10 == 0:
        elapsed = time() - start_time
        print(f"{i + 1}/{total_rows} 완료 ({elapsed:.1f}초 경과)")


df["document_similarity"] = all_doc_sims
df["mean_document_similarity"] = all_mean_sims

df.to_csv("./results/retrieval_data_score_v2.csv", index=False, encoding="utf-8")

총 50개 row 평가 시작
10/50 완료 (78.3초 경과)
20/50 완료 (143.6초 경과)
30/50 완료 (200.9초 경과)
40/50 완료 (263.8초 경과)
50/50 완료 (327.6초 경과)
🎉 평가 완료! CSV 저장됨.


In [15]:
df["mean_document_similarity"].mean()

np.float64(0.7208)

### v1: 0.7296
### v2: 0.7208
### v3: 0.7272
### 0.7259 ± 0.0046
